In [181]:
import numpy as np
import tensorflow as tf 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras as keras
from tensorflow.keras.layers import Concatenate, Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Flatten
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Convolution1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

In [182]:
review_df = pd.read_pickle('food_review_places.pickle') #275568 observations

In [183]:
len(review_df.index)

70195

In [184]:
review_df = review_df[review_df['rating']!=0]

In [185]:
review_df['rating'].value_counts()

5.0    34487
4.0    19041
3.0     8118
2.0     5010
1.0     3539
Name: rating, dtype: int64

In [186]:
enc = OneHotEncoder()
reviews = list(review_df['reviewText'])
ratings = np.array(list(review_df['rating']))
# one-hot encode the ratings
ratings = enc.fit_transform(ratings.reshape(-1, 1)).toarray()

In [187]:
split = 0.8
train_len = int(split * len(reviews))
train_reviews = np.array(reviews[:train_len])
train_ratings = np.array(ratings[:train_len])
test_reviews = np.array(reviews[train_len:])
test_ratings = np.array(ratings[train_len:])

In [188]:
max_size = 1000
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_size, output_mode='int', output_sequence_length=14)
vectorize_layer.adapt(train_reviews)

D:\Anaconda\envs\py36gpu\lib\site-packages\tensorflow\python\keras\layers\preprocessing\text_vectorization.py:338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(preprocessed_data.to_list())


In [189]:
inputs = tf.keras.layers.Input(shape=(1, ), dtype=tf.string, name='text')
outputs = vectorize_layer(inputs)
model = tf.keras.Model(inputs, outputs)
# vocab = np.array(vectorize_layer.get_vocabulary())

In [190]:
encoded_train_reviews = model.predict(train_reviews)
encoded_test_reviews = model.predict(test_reviews)

#### Attention 

In [191]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
 
    def call(self, features, hidden):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = tf.nn.tanh(
            self.W1(features) + self.W2(hidden_with_time_axis))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
 
        return context_vector, attention_weights

#### Model

In [192]:
input_layer = Input(shape=(14,), dtype="int32")
layer1 = Embedding(input_dim=1001, output_dim=128, mask_zero=True)(input_layer)
lstm = Bidirectional(LSTM(10, return_sequences = True), name="bi_lstm_0")(layer1)
(lstm, forward_h, forward_c, backward_h, backward_c) = Bidirectional(LSTM(128, return_sequences=True, return_state=True), name="bi_lstm_1")(lstm)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

context_vector, attention_weights = Attention(10)(lstm, state_h)

dense1 = Dense(128, activation='relu')(context_vector)
dense2 = Dense(32, activation='relu')(dense1)

dropout = Dropout(0.1)(dense2)
output = Dense(5, activation='softmax')(dropout)

In [193]:
 model = tf.keras.Model(input_layer, output)

In [194]:
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 14)]         0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, 14, 128)      128128      input_12[0][0]                   
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectional)       (None, 14, 20)       11120       embedding_18[0][0]               
__________________________________________________________________________________________________
bi_lstm_1 (Bidirectional)       [(None, 14, 256), (N 152576      bi_lstm_0[0][0]                  
___________________________________________________________________________________________

In [195]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(encoded_train_reviews, train_ratings, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
1404/1404 [==============================] - 98s 69ms/step - loss: 1.0297 - accuracy: 0.5483 - val_loss: 0.9848 - val_accuracy: 0.5670
Epoch 2/10
1404/1404 [==============================] - 97s 69ms/step - loss: 0.9477 - accuracy: 0.5826 - val_loss: 0.9696 - val_accuracy: 0.5744
Epoch 3/10
1404/1404 [==============================] - 95s 68ms/step - loss: 0.9288 - accuracy: 0.5918 - val_loss: 0.9776 - val_accuracy: 0.5723
Epoch 4/10
1404/1404 [==============================] - 97s 69ms/step - loss: 0.9120 - accuracy: 0.5975 - val_loss: 0.9764 - val_accuracy: 0.5731
Epoch 5/10
1404/1404 [==============================] - 99s 70ms/step - loss: 0.8989 - accuracy: 0.6040 - val_loss: 0.9836 - val_accuracy: 0.5668
Epoch 6/10
1404/1404 [==============================] - 96s 68ms/step - loss: 0.8826 - accuracy: 0.6110 - val_loss: 0.9954 - val_accuracy: 0.5710
Epoch 7/10
1404/1404 [==============================] - 95s 68ms/step - loss: 0.8656 - accuracy: 0.6166 - val_loss: 1.0054 -

In [101]:
score, acc = model.evaluate(encoded_test_reviews, test_ratings)

1723/1723 [==============================] - 5s 3ms/step - loss: 0.9652 - accuracy: 0.5706
